In [ ]:
# 1. THE ULTIMATE CLEANUP
print("🧹 Wiping system for a fresh start...")
!pkill -9 dockerd || true
!pkill -9 cloudflared || true
!rm -rf /etc/apt/sources.list.d/r2u.list

# 2. INSTALL DOCKER ENGINE
print("🏗️ Installing Engine...")
!apt-get update -qq > /dev/null
!apt-get install -y docker.io adb -qq > /dev/null

# 3. START ENGINE MANUALLY
print("⚡ Starting Motor...")
import os, subprocess, time, threading
get_ipython().system_raw('dockerd --storage-driver=vfs --host=unix:///var/run/docker.sock &')
time.sleep(15)

# 4. LAUNCH ANDROID WITH INTEGRATED WEB VIEWER
print("📱 Booting Android + Web Interface...")
!docker run -d --privileged --name android -p 5555:5555 redroid/redroid:11.0.0-latest > /dev/null
!docker run -d --privileged --name viewer --link android -p 7860:8080 -e A_HOST=android -e A_PORT=5555 freddy00/redroid-web > /dev/null
time.sleep(10)

# 5. DOWNLOAD & RUN CLOUDFLARE DIRECTLY
print("🔗 CREATING TUNNEL (NO LOGIN REQUIRED)...")
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O cloudflared
!chmod +x cloudflared

def start_tunnel():
    os.system('./cloudflared tunnel --url http://localhost:7860')

threading.Thread(target=start_tunnel, daemon=True).start()
time.sleep(15)

print("\n" + "*"*60)
print("🚀 SUCCESS! SCROLL DOWN AND LOOK FOR THE '.trycloudflare.com' LINK")
print("*"*60)